In [7]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("course_lead_scoring.csv")

In [5]:
df.columns

Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score',
       'converted'],
      dtype='object')

In [22]:
# check if the missing values are presented in the features.
# If there are missing values:
# For caterogiral features, replace them with 'NA'
# For numerical features, replace with with 0.0
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [23]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [32]:
categorical = ['lead_source', 'industry', 'employment_status', 'location']
numeric = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score','converted']

for category in categorical:
    df[category] = df[category].fillna('NA')

for num in numeric:
    df[num] = df[num].fillna(0.0)

df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [25]:
df.industry.value_counts()

industry
retail           203
finance          200
other            198
healthcare       187
education        187
technology       179
manufacturing    174
NA               134
Name: count, dtype: int64

In [13]:
# interaction_count and lead_score
# number_of_courses_viewed and lead_score
df[['interaction_count', 'number_of_courses_viewed']].corrwith(df.lead_score)

interaction_count           0.009888
number_of_courses_viewed   -0.004879
dtype: float64

In [14]:
# number_of_courses_viewed and interaction_count
# annual_income and interaction_count
df[['annual_income', 'number_of_courses_viewed']].corrwith(df.interaction_count)

annual_income               0.048618
number_of_courses_viewed   -0.023565
dtype: float64

In [33]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [34]:
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [35]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [36]:
y_train = df_train.lead_score.values
y_val = df_val.lead_score.values
y_test = df_test.lead_score.values

In [37]:
del df_train['lead_score']
del df_val['lead_score']
del df_test['lead_score']

In [38]:
from sklearn.metrics import mutual_info_score

In [39]:
# Calculate the mutual information score between y and other categorical variables in the dataset. Use the training set only.
# Round the scores to 2 decimals using round(score, 2).
def mutual_info_lead_score(series):
    return mutual_info_score(series, y_train)

mi = df_train[categorical].apply(mutual_info_lead_score)
mi.sort_values(ascending=False)

/Users/prograsshopper/Library/Caches/pypoetry/virtualenvs/ml-zoomcamp-homework-H2aynh9Z-py3.13/lib/python3.13/site-packages/sklearn/metrics/cluster/_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received multiclass values for label, and continuous values for target
  warnings.warn(msg, UserWarning)
/Users/prograsshopper/Library/Caches/pypoetry/virtualenvs/ml-zoomcamp-homework-H2aynh9Z-py3.13/lib/python3.13/site-packages/sklearn/metrics/cluster/_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received multiclass values for label, and continuous values for target
  warnings.warn(msg, UserWarning)
/Users/prograsshopper/Library/Caches/pypoetry/virtualenvs/ml-zoomcamp-homework-H2aynh9Z-py3.13/lib/python3.13/site-packages/sklearn/metrics/cluster/_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received multiclass values for label, and continuous values for target
  warnings.warn(msg, UserWarning)
/Use

industry             0.513917
location             0.435807
lead_source          0.330194
employment_status    0.273385
dtype: float64

In [40]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# Fit the model on the training dataset.
# To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
# model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
# Calculate the accuracy on the validation dataset and round it to 2 decimal digits.
